In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default
import itertools

In [ ]:
# Authenticate and create the gspread client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# STEP 1 - Load the Google Sheet (Published as CSV)
# ---------------------------------------------
# Replace with the name or URL of your Google Sheet
csv_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRF-FqJhbr-E4YGV2v43HZBFD009FR6wDBDgZIxzUWEv45omwyA_DG8dGdjXYoF_c355AA8h7s-4RVx/pub?gid=0&single=true&output=csv"

# Read the Sheet as real values
typology_df = pd.read_csv(csv_url)

# Preview the raw data
print("✅ Raw Typology Data Loaded")
print(typology_df.head(20))

✅ Raw Typology Data Loaded
    Partition  Insulation  Facade               Name Code Name  Thickness (m)  \
0         0.0         0.0     1.0     Concrete 30/37       CON        0.18000   
1         NaN         NaN     NaN     Concrete 20/25       CON        0.16000   
2         NaN         NaN     NaN     Concrete 25/30       CON        0.16000   
3         NaN         NaN     NaN     Concrete 25/30       CON        0.18000   
4         0.0         NaN     0.0              Earth       EAR        0.50000   
5         0.0         NaN     0.0              Earth       EAR        0.30000   
6         NaN         NaN     NaN              Straw       STR        0.40000   
7         1.0         NaN     0.0  Timber Mass (CLT)       TIM        0.24000   
8         NaN         NaN     NaN  Timber Mass (CLT)       TIM        0.22000   
9         NaN         NaN     NaN  Timber Mass (CLT)       TIM        0.14000   
10        1.0         0.0     0.0       Timber frame       TIF        0.12000   
1

In [ ]:
# STEP 2 - Clean Typology Data
# -----------------------------------------------------------------
# Keep necessary columns
typology_clean = typology_df[['Name', 'Code Name', 'Thickness (m)', 'EW (Ext. Wall)',
                              'IW (Int. Wall)', 'ES (Ext.Slab)', 'IS (Int. Slab)',
                              'RO (Roof)', 'A-D/m²']]

# Rename for easier handling
typology_clean.columns = ['Name', 'Code', 'Thickness', 'EW', 'IW', 'ES', 'IS', 'RO', 'A-D']

# Drop missing important values
typology_clean = typology_clean.dropna(subset=['Code', 'Thickness', 'A-D'])

# Convert columns to correct types
for col in ['Thickness', 'EW', 'IW', 'ES', 'IS', 'RO', 'A-D']:
    typology_clean[col] = typology_clean[col].astype(float)

print("✅ Typology Data Cleaned")
print(typology_clean.head(20))


✅ Typology Data Cleaned
                 Name Code  Thickness   EW   IW   ES   IS   RO        A-D
0      Concrete 30/37  CON    0.18000  0.0  0.0  1.0  0.0  1.0  34.882920
1      Concrete 20/25  CON    0.16000  0.0  1.0  0.0  0.0  0.0  24.671040
2      Concrete 25/30  CON    0.16000  0.0  0.0  0.0  1.0  0.0  28.543040
3      Concrete 25/30  CON    0.18000  1.0  0.0  0.0  0.0  0.0  32.110920
4               Earth  EAR    0.50000  1.0  0.0  0.0  0.0  0.0  11.803700
5               Earth  EAR    0.30000  0.0  1.0  0.0  0.0  0.0   7.082220
6               Straw  STR    0.40000  1.0  1.0  0.0  0.0  0.0  10.516800
7   Timber Mass (CLT)  TIM    0.24000  0.0  0.0  0.0  1.0  1.0  -2.784720
8   Timber Mass (CLT)  TIM    0.22000  1.0  0.0  0.0  0.0  0.0  -2.552660
9   Timber Mass (CLT)  TIM    0.14000  0.0  1.0  0.0  0.0  0.0  -1.624420
10       Timber frame  TIF    0.12000  1.0  0.0  0.0  0.0  0.0  -0.067637
11       Timber frame  TIF    0.08000  0.0  1.0  0.0  0.0  0.0  -0.052607
12       Timbe

In [ ]:
# STEP 3 - Prepare Material Categories
# -----------------------------------------------------------------
materials = {
    'EW': typology_clean[typology_clean['EW'] == 1],
    'IW': typology_clean[typology_clean['IW'] == 1],
    'ES': typology_clean[typology_clean['ES'] == 1],
    'IS': typology_clean[typology_clean['IS'] == 1],
    'RO': typology_clean[typology_clean['RO'] == 1]
}

# Validate number of options per element
print(f"Number of EW options: {len(materials['EW'])}")
print(f"Number of IW options: {len(materials['IW'])}")
print(f"Number of ES options: {len(materials['ES'])}")
print(f"Number of IS options: {len(materials['IS'])}")
print(f"Number of RO options: {len(materials['RO'])}")

# Calculate expected total combinations
total_combinations = (
    len(materials['EW']) *
    len(materials['IW']) *
    len(materials['ES']) *
    len(materials['IS']) *
    len(materials['RO'])
)

print(f"Expected total number of construction typologies: {total_combinations}")



Number of EW options: 6
Number of IW options: 6
Number of ES options: 1
Number of IS options: 3
Number of RO options: 3
Expected total number of construction typologies: 324


In [ ]:
# STEP 4 - Generate Typology Combinations (NO dummy areas anymore)
# -----------------------------------------------------------------
combinations = list(itertools.product(
    materials['EW'].index,
    materials['IW'].index,
    materials['ES'].index,
    materials['IS'].index,
    materials['RO'].index
))

results = []

for combo in combinations:
    elements = {
        'EW': typology_clean.loc[combo[0]],
        'IW': typology_clean.loc[combo[1]],
        'ES': typology_clean.loc[combo[2]],
        'IS': typology_clean.loc[combo[3]],
        'RO': typology_clean.loc[combo[4]]
    }

    code_parts = []
    name_parts = []

    for element_type, material in elements.items():
        code_parts.append(material['Code'])
        name_parts.append(material['Name'])

    combo_code = '_'.join(code_parts)

    results.append({
        "Combination": ' | '.join(name_parts),
        "Code": combo_code,
        "EW_Code": elements['EW']['Code'],
        "IW_Code": elements['IW']['Code'],
        "ES_Code": elements['ES']['Code'],
        "IS_Code": elements['IS']['Code'],
        "RO_Code": elements['RO']['Code'],
        "EW_A-D/m²": elements['EW']['A-D'],
        "IW_A-D/m²": elements['IW']['A-D'],
        "ES_A-D/m²": elements['ES']['A-D'],
        "IS_A-D/m²": elements['IS']['A-D'],
        "RO_A-D/m²": elements['RO']['A-D'],
    })

# Create result_df
result_df = pd.DataFrame(results)

# Remove duplicates if necessary
result_df = result_df.drop_duplicates(subset=['Code'])

print("✅ Typology Combinations Created (no area multiplication yet)")
print(result_df.head())


✅ Typology Combinations Created (no area multiplication yet)
                                         Combination                 Code  \
0  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
1  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_TIM   
2  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_TIF   
3  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_TIM_CON   
4  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_TIM_TIM   

  EW_Code IW_Code ES_Code IS_Code RO_Code  EW_A-D/m²  IW_A-D/m²  ES_A-D/m²  \
0     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   
1     CON     CON     CON     CON     TIM   32.11092   24.67104   34.88292   
2     CON     CON     CON     CON     TIF   32.11092   24.67104   34.88292   
3     CON     CON     CON     TIM     CON   32.11092   24.67104   34.88292   
4     CON     CON     CON     TIM     TIM   32.11092   24.67104   34.88292   

   IS_A

In [ ]:
# STEP 5 - Load the Building Data CSV (Areas per SUM Iteration)
# -----------------------------------------------------------------
bdg_df = pd.read_csv('/content/drive/MyDrive/AIA_DataEncoding/BDG_v4.csv')

print("✅ Raw Building Data Loaded")
print(bdg_df.head())

# Filter only 'SUM' rows, keep ALL columns
sum_rows_df = bdg_df[bdg_df['ITERATION'] == 'SUM'].copy()

# Check
print("✅ SUM Rows with all metadata extracted")
print(sum_rows_df.head())

✅ Raw Building Data Loaded
  ITERATION                                        ID  LEVEL  USABLE_AREA  \
0     BLD_3  50_70_0_12_1500_3000_True_0.3_36_0_40_20      0            0   
1     BLD_3  50_70_0_12_1500_3000_True_0.3_36_0_40_20      1          144   
2     BLD_3  50_70_0_12_1500_3000_True_0.3_36_0_40_20      2          126   
3     BLD_3  50_70_0_12_1500_3000_True_0.3_36_0_40_20      3            0   
4     BLD_3  50_70_0_12_1500_3000_True_0.3_36_0_40_20      4            0   

   TOTAL_AREA  1BD  2BD  3BD  N_WALLS  WALLS_M2  ...  N_INT_SLABS  \
0           0    0    0    0     56.0     183.0  ...          NaN   
1         501    6   12    0    144.0     713.0  ...          NaN   
2         348    6   10    0    120.0     620.0  ...          NaN   
3           0    0    0    0     44.0     144.0  ...          NaN   
4           0    0    0    0      NaN       NaN  ...          NaN   

   INT_SLABS_M2  N_WINDOWS  WINDOWS_M2  N_CORR_WINDOWS  CORR_WINDOWS_M2  \
0           NaN     

In [ ]:
# STEP 6 - Cross Join Typology Results with SUM Building Data
# -----------------------------------------------------------------
result_df['_tmpkey'] = 1
sum_rows_df['_tmpkey'] = 1

ML_df = pd.merge(result_df, sum_rows_df, on='_tmpkey').drop('_tmpkey', axis=1)

print("✅ Cross-Joined ML_df Created (typologies × building SUM data)")
print(ML_df.head())

✅ Cross-Joined ML_df Created (typologies × building SUM data)
                                         Combination                 Code  \
0  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
1  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
2  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
3  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
4  Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   

  EW_Code IW_Code ES_Code IS_Code RO_Code  EW_A-D/m²  IW_A-D/m²  ES_A-D/m²  \
0     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   
1     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   
2     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   
3     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   
4     CON     CON     CON     CON     CON   32.11092   24.67104   34.88292   

   ...

In [ ]:
# STEP 7 - Recalculate Corrected Weighted A-D using Real Areas
# -----------------------------------------------------------------
ML_df['Corrected Total Weighted A-D'] = (
    ML_df['EW_A-D/m²'] * ML_df['EXT_WALL_M2'] +
    ML_df['IW_A-D/m²'] * ML_df['INT_WALLS_M2'] +
    ML_df['ES_A-D/m²'] * ML_df['EXT_SLABS_M2'] +
    ML_df['IS_A-D/m²'] * ML_df['INT_SLABS_M2'] +
    ML_df['RO_A-D/m²'] * ML_df['ROOF_M2']
)

# STEP 7.1 - Normalize A-D using existing TOTAL_AREA column
ML_df['Normalized A-D'] = ML_df['Corrected Total Weighted A-D'] / ML_df['TOTAL_AREA']
print("✅ Normalized A-D calculated using TOTAL_AREA")

print(ML_df['Normalized A-D'].head())
print(ML_df['Normalized A-D'].size)

✅ Normalized A-D calculated using TOTAL_AREA
0    124.949429
1    141.411417
2    144.278660
3    149.658653
4    128.020639
Name: Normalized A-D, dtype: float64
12636


In [ ]:
# STEP 8 - Round all numerical values to 2 decimals
# -----------------------------------------------------------------
# Round only float columns
for col in ML_df.select_dtypes(include=['float']):
    ML_df[col] = ML_df[col].round(2)

print("✅ Rounded all numeric values to 2 decimals")

✅ Rounded all numeric values to 2 decimals


In [ ]:
# STEP 9 - Save Outputs
# -----------------------------------------------------------------
ML_df.to_csv('ML_df_ready.csv', index=False)

print("✅ Final ML_df saved as 'ML_df_ready.csv'")

# Optional Download Button
from google.colab import files
files.download('ML_df_ready.csv')

✅ Final ML_df saved as 'ML_df_ready.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>